# Support Vector Machines Part 1

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn import svm
from sklearn import preprocessing

In [ ]:
%config InlineBackend.figure_formats =['svg']

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

cmap = mpl.colors.ListedColormap(['red', 'black', 'blue'])

## Task 1 Decision Boundary from a Maximal Margin Classifier (6 points)
Let's create some synthetic data with two classes:

In [ ]:
X, y = datasets.make_blobs(n_samples=50, centers=2,
                           random_state=0, cluster_std=0.60)

y[np.where(y == 0)] = -1

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap=cmap)

ax.set_xlabel(r'$x_1$', fontsize=16)
ax.set_ylabel(r'$x_2$', fontsize=16)

plt.show()

In this task we will use the `SVC` model of `sklearn.svm` with a `kernel=linear` and large parameter `C`. `sklearn`'s module `svm` has implementations of several Support Vector Machine algorithms. Some are meant to solve Regression, some - Classification problems. `LinearSVC` stands for Linear Support Vector Classification which is exactly what we need in order to solve the maximal margin classification problem. However, `svm.SVC` offers more flexibility in order to extract the learned parameters etc.

In [ ]:
model = svm.SVC(kernel='linear', C=1000, tol=0.0001)  # initialize the model
model.fit(X, y)  # fit the model = learn the decision boundaries

When the optimisation has completed, you can access the parameters of the separating hyperplane. The vector $\mathbf{w}$:

In [ ]:
model.coef_

The $b$:

In [ ]:
model.intercept_

The indices of the support vectors:

In [ ]:
model.support_

So you can slice $\mathbf{X}$ for the corresponding coordinates

In [ ]:
X[model.support_]

or get them directly:

In [ ]:
model.support_vectors_

**Task 1A** Calculate $y^{(m)}\left(b+w^Tx^{(m)}\right)$ for every support vector. Please explain whether the result complies with your expectation and why?

In [ ]:
mag = []
for idx in model.support_:
    mag.append(y[idx] * (model.intercept_ + model.coef_ @ X[idx].T)[0])

print(mag)
print([y[idx] for idx in model.support_])

## 📢 **HAND-IN** 📢: Report the calculated values and your explanation from Task 1A in Moodle

In [ ]:
model.predict(X)

**Task 1B** Calculate the distance from the decision boundary to the support vectors, i.e. the width of the margin

In [ ]:
mag = []
w_sig = np.linalg.norm(model.coef_)
for idx in model.support_:
    mag.append(y[idx] / w_sig * (model.intercept_ + model.coef_ @ X[idx].T)[0])

print(mag)

## 📢 **HAND-IN** 📢: Report the width of the margin calculated in Task 1B in Moodle

**Task 1C** Generate a scatterplot with the data $X$, the decision boundary and the gutters like in
https://github.zhaw.ch/pages/doem/mldm_book/06_svm.html#fig-max-margin
using the optimised parameters `model.interecept_` and `model.coef_[0]`

## 📢 **HAND-IN** 📢: Upload an image-file of the plot from task 1C in Moodle

In [ ]:
def plot_scatter():
    from sklearn.inspection import DecisionBoundaryDisplay

    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

    # plot the decision function
    ax = plt.gca()
    DecisionBoundaryDisplay.from_estimator(
        model,
        X,
        plot_method="contour",
        colors="k",
        levels=[-1, 0, 1],
        alpha=0.5,
        linestyles=["--", "-", "--"],
        ax=ax,
    )
    # plot support vectors
    ax.scatter(
        model.support_vectors_[:, 0],
        model.support_vectors_[:, 1],
        s=100,
        linewidth=1,
        facecolors="none",
        edgecolors="k",
    )

In [ ]:
  def plot_scatter(clf):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)
    # plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = clf.decision_function(xy).reshape(XX.shape)
    ax.scatter(XX, YY, marker=".", alpha=0.5, color='gray')
    # plot decision boundary and margins
    ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    # plot support vectors
    ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
               linewidth=1, facecolors='none')

l## Task 2 Large Margin Classifier on the Wine Dataset (4 points)
This dataset `sklearn.datasets.load_wine()` contains the chemical analysis of 178 wine samples produced by 3 different cultivators: the goal is to train a classification model capable of predicting the cultivator based on the wine's chemical analysis. Since SVM classifiers are binary classifiers, you will need to use one-versus-all to classify all 3 classes. What accuracy can you reach?

In [ ]:
w = model.coef_[0]
w_norm = np.linalg.norm(w)
b = model.intercept_

for i in model.support_:
    xi = X[i]
    r = (b + w.T @ xi) / w_norm
    xi_T = xi - r * w / w_norm
    plt.plot([xi[0], xi_T[0]], [xi[1], xi_T[1]])
plot_scatter(model)
plt.savefig('foo.png')

In [3]:
wine = datasets.load_wine(as_frame=True)
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

**TASK 2A**: Train a linear SVM on the dataset and report the accuracy

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    wine.data, wine.target, random_state=42)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

SVC will automatically use the One-vs-All strategy, so there's nothing special we need to do to handle multiple classes.

In [5]:
model = svm.SVC(kernel='linear', C=1000, tol=0.0001, random_state=42)  # initialize the model


Calculate the accuracy (cross validation score):

In [6]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, X_test, y_test).mean()

0.9555555555555555

## 📢 **HAND-IN** 📢:
Report the accuracy that you reached in task 2A without feature scaling in Moodle

**Task 2B**: Which accuracy (cross validation score) can you reach when you standarize the features before training?  
See https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
cross_val_score(model, X_test, y_test).mean()

0.9777777777777779

## 📢 **HAND-IN** 📢: Report the accuracy that you reached with feature standaridsation in Task 2B in Moodle